In [1]:
import urllib, json
import datetime as dt
from urllib.request import urlopen
import numpy as np
import pandas as pd
import statistics

# Step 1 

## Data collection - collect Reddit user comment history

In [25]:
# You can manually create a list of users, and the script will collect their entire comment history from Reddit, 
# and write it file in a sub-directory called 'user_data'.

users = ['USER_1_NAME_HERE','USER_2_NAME_HERE','USER_N_NAME_HERE']

for name in users:
    sub_csv = open('user_data/' + name + '_submissions.csv', 'w')
    sub_csv.write("title, body, score, id, url, num_comments(approx.), created \n")
    total = 1001
    before_field = "0d"
    while(total > 1000):
        url = urlopen("https://api.pushshift.io/reddit/submission/search?metadata=true&before=" + before_field + "&limit=1000&sort=desc&author=" + name)
        # url = urlopen("https://api.pushshift.io/reddit/submission/search?subreddit=" + sub + "&metadata=true&before=" + before_field + "&limit=1000&sort=desc&author=" + name)
        user_data = json.loads(url.read().decode())
        sub_num = user_data["metadata"]["results_returned"]
        total = user_data["metadata"]["total_results"]
        for j in range(0, sub_num):
            submission = user_data["data"][j]
            sub_csv.write('' .join([i if ord(i) < 128 else ' ' for i in submission["title"].replace(",", "").replace("\n", ".")]) + ",")
            try:
                sub_csv.write('' .join([i if ord(i) < 128 else ' ' for i in submission["selftext"].replace(",", "").replace("\n", ".")]) + ",")
            except KeyError:
                sub_csv.write("[none],")
            sub_csv.write(str(submission["score"]) + ",")
            sub_csv.write('' .join([i if ord(i) < 128 else ' ' for i in submission["id"].replace(",", "").replace("\n", ".")]) + ",")
            sub_csv.write('' .join([i if ord(i) < 128 else ' ' for i in submission["url"].replace(",", "").replace("\n", ".")]) + ",")
            sub_csv.write('' .join([i if ord(i) < 128 else ' ' for i in str(submission["num_comments"]).replace(",", "").replace("\n", ".")]) + ",")
            sub_csv.write(dt.datetime.fromtimestamp(submission["created_utc"]).isoformat() + "\n")
            if(j == sub_num -1):
                before_field = str(user_data["data"][sub_num - 1]["created_utc"])
    sub_csv.close()

# Step 2

## Data already collected and saved to disk (Step 1 above). 

## Ready to import and analyse:

In [35]:
import os
# get a list of all the user data files 
user_files = os.listdir('user_data/')

In [49]:
# OPTIONAL - READ IN EXISTING FILE IF ALREADY PROCESSED
# final_results = pd.read_csv("final_results.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
user_files[0:5]

['CharlieDay77_comments.csv',
 'scubareddit12_comments.csv',
 'SUB_05_comments.csv',
 'SirRonaldofBurgundy_comments.csv',
 'RollAd20_comments.csv']

In [37]:
len(user_files)

9769

In [ ]:
# we will store all the data in this list:
votescore_and_mean_sentiment = []

# loop through all file names in the directory:
for user_filename in user_files:
    
    try:
        test_user_comments = pd.read_csv('user_data/' + user_filename)
        test_user_comments.columns = ['submission_id', 'parent_id', 'body', 'id', 'created', 'score']
        test_user_comments['created_fixed'] = pd.DatetimeIndex(test_user_comments['created'])
        test_user_comments['created_fixed_unix'] = test_user_comments['created_fixed'].astype(np.int64) // 10**9
    except:
        continue
        
    for index, row in test_user_comments.iterrows():
        temp_df = test_user_comments[(test_user_comments['created_fixed_unix'] < row['created_fixed_unix'] + (60*60)) & 
                       (test_user_comments['created_fixed_unix'] > row['created_fixed_unix'])]

        if(len(temp_df)==0):
            continue

        sentiment_scores = []

        for text in temp_df['body']:
            try:
                sentiment_scores.append(analyser.polarity_scores(text))
            except:
                sentiment_scores.append({'neg': 0, 'neu': 0, 'pos': 0, 'compound': 0})

        sentiment_compound_scores = [item['compound'] for item in sentiment_scores]

        node_id_temp = row['id']
        vote_score_temp = row['score']
        mean_sent_score = statistics.mean(sentiment_compound_scores)

        votescore_and_mean_sentiment.append({
            'id': node_id_temp, 
            'score': vote_score_temp,
            'mean_sentiment_score': mean_sent_score
        })

In [29]:
final_results = pd.DataFrame(votescore_and_mean_sentiment)

In [36]:
# write to file
final_results.to_csv(r'final_results.csv')